In [18]:
import os
import pandas as pd
import numpy as np
from src.config import get_project_root
#Rende il codice indipendente dal tuo filesystem local, a patto che scarichi la cartella 'fantacalcio'
ROOT = get_project_root()
#Create an object "calendar" and define following function as methods?

def import_calendar():
    #Funzione per importare il calendario sotto forma di dataframe
    data_path = os.path.join(ROOT,'data')
    calendar_raw_name = 'Calendario_Campionato---A-buon-rendere.xlsx'
    calendar_path = os.path.join(data_path,calendar_raw_name)
    if os.path.isfile(calendar_path):
        calendar_raw = pd.read_excel(calendar_path,skiprows=2)
    else:
        #Se non trova il calendario nella cartella 'data' si incazza
        print('porcoddio')
    return calendar_raw

def clean_calendar(calendar_raw):
    #Funzione per pulire il calendario e renderlo più utilizzabile.
    #Secondo me convienve fare un dataframe con ogni squadra e associato un vettore di 'Gol','FP','Avversari'
    #df_persquadra = pd.DataFrame(columns=['Squadre','Avversari','Gol', 'Fantapunti'])
    colonna_giornata = ['Unnamed: 0', 'Unnamed: 6']
    for cg in colonna_giornata:
        j = calendar_raw.columns.get_loc(cg)
        inizio_giornata = np.where(calendar_raw[cg].astype(str).str.contains('Giornata'))[0]
        for ig in inizio_giornata:
            nome_giornata = calendar_raw[cg][ig]
            ngiornata = int(''.join(x for x in nome_giornata if x.isdigit()))
            if ngiornata == 1:
                squadre = calendar_raw.iloc[ig+1:ig+5,j].append(calendar_raw.iloc[ig+1:ig+5,j+3],ignore_index=True)
                df_persquadra = pd.DataFrame(index=squadre)
                fp = calendar_raw.iloc[ig+1:ig+5,j+1].append(calendar_raw.iloc[ig+1:ig+5,j+2],ignore_index=True).astype(float)
                df_persquadra['Fantapunti'] = fp.values
                avversari = calendar_raw.iloc[ig+1:ig+5,j+3].append(calendar_raw.iloc[ig+1:ig+5,j],ignore_index=True)
                df_persquadra['Avversari'] = avversari.values
                Gol = np.zeros(8,dtype=int)
                results = calendar_raw.iloc[ig+1:ig+5,j+4]
                for idx,res in enumerate(results):
                    Gol[idx] = int(res.split('-')[0])
                    Gol[idx+4] = int(res.split('-')[1])
                df_persquadra['Gol'] = Gol
                df_persquadra= df_persquadra.astype(object)
                ordine_giornate = [ngiornata]
            else:
                #print(ngiornata,j)
                squadresx = calendar_raw.iloc[ig+1:ig+5,j]
                squadredx = calendar_raw.iloc[ig+1:ig+5,j+3]
                for idx,squadra in enumerate(squadresx):
                    #Per le squadre sulla sinistra il punteggio si trova alla loro destra
                    df_persquadra.at[squadra,'Fantapunti'] \
                    = np.append(df_persquadra.at[squadra,'Fantapunti'],calendar_raw.iloc[ig+idx+1,j+1])
                    df_persquadra.at[squadra,'Avversari'] \
                    = np.append(df_persquadra.at[squadra,'Avversari'],calendar_raw.iloc[ig+idx+1,j+3])
                    if calendar_raw.iloc[ig+idx+1,j+4].split('-')[0].isdigit():
                        gol = int(calendar_raw.iloc[ig+idx+1,j+4].split('-')[0])
                    else: 
                        gol = float('NaN')
                    df_persquadra.at[squadra,'Gol'] \
                    = np.append(df_persquadra.at[squadra,'Gol'],gol)
                for idx,squadra in enumerate(squadredx):
                    #Per le squadre sulla sinistra il punteggio si trova alla loro destra
                    df_persquadra.at[squadra,'Fantapunti'] \
                    = np.append(df_persquadra.at[squadra,'Fantapunti'],calendar_raw.iloc[ig+idx+1,j+2])
                    df_persquadra.at[squadra,'Avversari'] \
                    = np.append(df_persquadra.at[squadra,'Avversari'],calendar_raw.iloc[ig+idx+1,j])
                    if calendar_raw.iloc[ig+idx+1,j+4].split('-')[1].isdigit():
                        gol = int(calendar_raw.iloc[ig+idx+1,j+4].split('-')[1])
                    else: 
                        gol = float('NaN')
                    df_persquadra.at[squadra,'Gol'] \
                    = np.append(df_persquadra.at[squadra,'Gol'],gol)
              
                ordine_giornate.append(ngiornata)
    srtidx = np.argsort(ordine_giornate)
    for squadra in df_persquadra.index:
        df_persquadra.loc[squadra].Fantapunti=\
        df_persquadra.loc[squadra].Fantapunti[srtidx]
        df_persquadra.loc[squadra].Avversari =\
        df_persquadra.loc[squadra].Avversari[srtidx]
        df_persquadra.loc[squadra].Gol =\
        df_persquadra.loc[squadra].Gol[srtidx]
    return  df_persquadra,np.array(ordine_giornate)

In [19]:
calendar_raw = import_calendar()
df_persquadra,ordine_giornate = clean_calendar(calendar_raw)

In [20]:
df_persquadra

,Fantapunti,Avversari,Gol
Gargantua,"[64.0, 66.5, 77.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[Capitan Martella, CACCOLA F.C., Barriera-team...","[0.0, 1.0, 3.0, nan, nan, nan, nan, nan, nan, ..."
asdMalerba,"[74.0, 86.0, 66.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[CACCOLA F.C., Paris FC, SamPDORRR FC, Capitan...","[3.0, 6.0, 1.0, nan, nan, nan, nan, nan, nan, ..."
Civediamoamaggio,"[70.5, 75.5, 78.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[Paris FC, SamPDORRR FC, Capitan Martella, Gar...","[2.0, 3.0, 4.0, nan, nan, nan, nan, nan, nan, ..."
Barriera-team,"[70.0, 75.0, 70.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[SamPDORRR FC, Capitan Martella, Gargantua , C...","[2.0, 3.0, 2.0, nan, nan, nan, nan, nan, nan, ..."
Capitan Martella,"[83.0, 74.5, 62.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[Gargantua , Barriera-team, Civediamoamaggio ,...","[5.0, 3.0, 0.0, nan, nan, nan, nan, nan, nan, ..."
CACCOLA F.C.,"[79.0, 72.0, 77.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[asdMalerba, Gargantua , Paris FC, Barriera-te...","[4.0, 2.0, 3.0, nan, nan, nan, nan, nan, nan, ..."
Paris FC,"[70.5, 78.5, 70.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[Civediamoamaggio , asdMalerba, CACCOLA F.C., ...","[2.0, 4.0, 2.0, nan, nan, nan, nan, nan, nan, ..."
SamPDORRR FC,"[83.5, 82.5, 73.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[Barriera-team, Civediamoamaggio , asdMalerba,...","[5.0, 5.0, 2.0, nan, nan, nan, nan, nan, nan, ..."


In [7]:
calendar_raw

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1ª Giornata lega,NaN,4ª Giornata serie a,NaN,NaN,NaN,2ª Giornata lega,NaN,5ª Giornata serie a,NaN,NaN
1,Gargantua,64.0,83,Capitan Martella,0-5,NaN,Capitan Martella,74.5,75,Barriera-team,3-3
2,asdMalerba,74.0,79,CACCOLA F.C.,3-4,NaN,SamPDORRR FC,82.5,75.5,Civediamoamaggio,5-3
3,Civediamoamaggio,70.5,70.5,Paris FC,2-2,NaN,Paris FC,78.5,86,asdMalerba,4-6
4,Barriera-team,70.0,83.5,SamPDORRR FC,2-5,NaN,CACCOLA F.C.,72.0,66.5,Gargantua,2-1
5,3ª Giornata lega,NaN,6ª Giornata serie a,NaN,NaN,NaN,4ª Giornata lega,NaN,7ª Giornata serie a,NaN,NaN
6,asdMalerba,66.0,73.5,SamPDORRR FC,1-2,NaN,Gargantua,0.0,0,Civediamoamaggio,-
7,Civediamoamaggio,78.5,62.5,Capitan Martella,4-0,NaN,Capitan Martella,0.0,0,asdMalerba,-
8,Barriera-team,70.5,77.5,Gargantua,2-3,NaN,SamPDORRR FC,0.0,0,Paris FC,-
9,Paris FC,70.5,77.5,CACCOLA F.C.,2-3,NaN,CACCOLA F.C.,0.0,0,Barriera-team,-


In [10]:
srtidx = np.argsort(ordine_giornate)